<a href="https://colab.research.google.com/github/biaferre/bioinspirada/blob/main/8_Rainhas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Implementação Base

### Definição de parâmetros

In [ ]:
import random

N = 8
start_mutation_rate = 0.4
start_crossover_rate = 0.9
start_population_size = 5
start_generations_max = 100

### Funções auxiliares
Aqui temos


*   Geração de indivíduos pra população inicial
*   Avaliação de fitness
*   Seleção de pais por torneio
*   Recombinação cut e crossfill
*   Mutação por troca de gene

In [ ]:
def generate_individual():
    return random.sample(range(N), N)

def fitness(individual):
    non_attacking = 28
    for i in range(N):
        for j in range(i + 1, N):
            if abs(individual[i] - individual[j]) == abs(i - j):
                non_attacking -= 1
    return non_attacking

def tournament_selection(population, k=5):
    return max(random.sample(population, k), key=fitness)

def cut_and_crossfill(parent1, parent2, crossover_rate):
    if random.random() < crossover_rate:
      cut = random.randint(1, N - 1)
      child1 = parent1[:cut]
      child2 = parent2[:cut]

      fill1 = [gene for gene in parent2 if gene not in child1]
      fill2 = [gene for gene in parent1 if gene not in child2]

      return child1 + fill1, child2 + fill2

    else:
      return parent1, parent2

def mutate(individual, mutation_rate):
    if random.random() < mutation_rate:
        i, j = random.sample(range(N), 2)
        individual[i], individual[j] = individual[j], individual[i]

### Algoritmo Genético em execução

In [ ]:
def genetic_algorithm(population_size, generations_max, crossover_rate, mutation_rate):
    population = [generate_individual() for _ in range(population_size)]
    for generation in range(generations_max):
        offspring = []
        while len(offspring) < population_size:
            parent1 = tournament_selection(population)
            parent2 = tournament_selection(population)
            child1, child2 = cut_and_crossfill(parent1, parent2, crossover_rate)
            mutate(child1, mutation_rate)
            mutate(child2, mutation_rate)
            offspring.extend([child1, child2])

        combined_population = population + offspring
        combined_population.sort(key=fitness, reverse=True)
        population = combined_population[:population_size]

        if fitness(population[0]) == 28:
            print(f"Solução encontrada na geração {generation}: {population[0]}")
            return population[0]

    print("Melhor solução encontrada:", population[0])
    return population[0]

    print("Melhor solução encontrada:", population[0])
    return population[0]

solution = genetic_algorithm(start_population_size, start_generations_max, start_crossover_rate, start_mutation_rate)

Melhor solução encontrada: [5, 0, 2, 6, 3, 1, 7, 4]


Visualizando o tabuleiro:

In [ ]:
if solution:
    board = [["." for _ in range(N)] for _ in range(N)]
    for col, row in enumerate(solution):
      board[row][col] = "Q"
    print("\nTabuleiro Final:")
    for row in board:
      print(" ".join(row))


Tabuleiro Final:
. Q . . . . . .
. . . . . Q . .
. . Q . . . . .
. . . . Q . . .
. . . . . . . Q
Q . . . . . . .
. . . Q . . . .
. . . . . . Q .


## Melhorias & Análise
Para explorar possíveis melhorias no algoritmo, podemos atacar por frentes diferentes:

* Taxas de mutação e recombinação
* Tamanho da população
* Método de seleção de pais
* Método de seleção de sobreviventes

Vamos usar os seguintes critérios de análise:

1. Execuções necessárias pra convergir
2. Média e desvio padrão
3. Nº de indivíduos que convergiram por execução
4. Fitness médio x Fitness mais alto por execução
5. Fitness médio total

Editando o código pra incluir esses critérios:

In [ ]:
def genetic_algorithm(population_size, generations_max, crossover_rate, mutation_rate):
    population = [generate_individual() for _ in range(population_size)]
    result = {}
    for generation in range(generations_max):
        offspring = []
        while len(offspring) < population_size:
            parent1 = tournament_selection(population)
            parent2 = tournament_selection(population)
            child1, child2 = cut_and_crossfill(parent1, parent2, crossover_rate)
            mutate(child1, mutation_rate)
            mutate(child2, mutation_rate)
            offspring.extend([child1, child2])

        combined_population = population + offspring
        combined_population.sort(key=fitness, reverse=True)
        population = combined_population[:population_size]

        if fitness(population[0]) == 28:
            print(f"Solução encontrada na geração {generation}: {population[0]}")
            result = {
                "solution": population[0],
                "generations": generation,
                "fitness": fitness(population[0]),
                "converged": True
            }
            return result

    result = {
          "solution": population[0],
          "generations": generations_max,
          "fitness": fitness(population[0]),
          "converged": False
    }

    return result

In [ ]:
import statistics
import pandas as pd
from collections import defaultdict

def run_experiments():
    mutation_rates = [0.0001, 0.001, 0.1, 0.3, 0.5]
    crossover_rates = [0.001, 0.1, 0.6, 0.8, 0.9]
    population_sizes = [10, 20, 50, 100, 200]
    generations_max = 1000
    runs_per_config = 10

    results = []

    all_data = []

    for pop_size in population_sizes:
        for mut_rate in mutation_rates:
            for cross_rate in crossover_rates:
                fitnesses = []
                generations = []
                converged_count = 0

                for _ in range(runs_per_config):
                    result = genetic_algorithm(
                        pop_size, generations_max, cross_rate, mut_rate
                    )
                    fitnesses.append(result["fitness"])
                    generations.append(result["generations"])
                    if result["converged"]:
                        converged_count += 1

                config = {
                    "population": pop_size,
                    "mutation_rate": mut_rate,
                    "crossover_rate": cross_rate,
                    "avg_fitness": statistics.mean(fitnesses),
                    "std_fitness": statistics.stdev(fitnesses),
                    "avg_generations": statistics.mean(generations),
                    "convergence_rate": converged_count / runs_per_config,
                    "fitnesses": fitnesses,
                    "generations": generations
                }

                results.append(config)
                all_data.append(config)

    df = pd.DataFrame(all_data)
    top5 = df.sort_values(by=["avg_fitness", "convergence_rate"], ascending=[False, False]).head(5)

    for i, r in top5.iterrows():
        print(f"Pop: {r['population']}, Mut: {r['mutation_rate']}, Cross: {r['crossover_rate']} | "
              f"Fitness: {r['avg_fitness']:.2f} ± {r['std_fitness']:.2f} | "
              f"Gens: {r['avg_generations']:.2f} | Conv: {int(r['convergence_rate'] * 100)}%")

In [ ]:
run_experiments()

Solução encontrada na geração 23: [5, 2, 6, 1, 7, 4, 0, 3]
Solução encontrada na geração 0: [5, 3, 6, 0, 2, 4, 1, 7]
Solução encontrada na geração 0: [2, 6, 1, 7, 5, 3, 0, 4]
Solução encontrada na geração 1: [2, 5, 1, 4, 7, 0, 6, 3]
Solução encontrada na geração 854: [6, 3, 1, 4, 7, 0, 2, 5]
Solução encontrada na geração 0: [2, 6, 1, 7, 4, 0, 3, 5]
Solução encontrada na geração 151: [6, 1, 3, 0, 7, 4, 2, 5]
Solução encontrada na geração 1: [7, 3, 0, 2, 5, 1, 6, 4]
Solução encontrada na geração 854: [4, 1, 5, 0, 6, 3, 7, 2]
Solução encontrada na geração 408: [5, 2, 0, 7, 4, 1, 3, 6]
Solução encontrada na geração 34: [2, 0, 6, 4, 7, 1, 3, 5]
Solução encontrada na geração 669: [1, 4, 6, 0, 2, 7, 5, 3]
Solução encontrada na geração 682: [5, 1, 6, 0, 3, 7, 4, 2]
Solução encontrada na geração 55: [3, 6, 4, 2, 0, 5, 7, 1]
Solução encontrada na geração 286: [2, 5, 7, 1, 3, 0, 6, 4]
Solução encontrada na geração 847: [1, 6, 4, 7, 0, 3, 5, 2]
Solução encontrada na geração 696: [5, 2, 6, 3, 0, 7,